In [1]:
import pandas as pd

In [2]:
DEATHS = pd.read_csv('covid_deaths_usafacts.csv')
CASES = pd.read_csv('covid_confirmed_usafacts.csv')
POPULATION = pd.read_csv('covid_county_population_usafacts.csv')

In [8]:
META_COLUMNS = ['countyFIPS', 'County Name', 'State', 'StateFIPS']

In [6]:
DEATHS

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
0,0,Statewide Unallocated,AL,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001,Autauga County,AL,1,0,0,0,0,0,0,...,235,235,235,235,235,235,235,235,235,235
2,1003,Baldwin County,AL,1,0,0,0,0,0,0,...,731,731,731,731,731,731,731,731,731,731
3,1005,Barbour County,AL,1,0,0,0,0,0,0,...,104,104,104,104,104,104,104,104,104,104
4,1007,Bibb County,AL,1,0,0,0,0,0,0,...,111,111,111,111,111,111,111,111,111,111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3188,56037,Sweetwater County,WY,56,0,0,0,0,0,0,...,142,142,142,142,142,142,142,142,142,142
3189,56039,Teton County,WY,56,0,0,0,0,0,0,...,16,16,16,16,16,16,16,16,16,16
3190,56041,Uinta County,WY,56,0,0,0,0,0,0,...,43,43,43,43,43,43,43,43,43,43
3191,56043,Washakie County,WY,56,0,0,0,0,0,0,...,51,51,51,51,51,51,51,51,51,51


In [ ]:
# This function converts CASES or DEATHS into a DataFrame containing totals on a given date
def national_data(frame):
    national_data = frame.drop(columns=META_COLUMNS, axis=1).sum()

    d = {'United States': national_data}

    df = pd.DataFrame(data=d).T

    return df

national_data(CASES)

,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,...,2023-07-14,2023-07-15,2023-07-16,2023-07-17,2023-07-18,2023-07-19,2023-07-20,2023-07-21,2023-07-22,2023-07-23
United States,723,734,741,751,759,769,775,779,786,802,...,99583860,99584700,99585122,99585659,99588872,99593865,99595974,99596741,99596741,99596741


In [ ]:
# This function converts CASES or DEATHS into a DataFrame organized by year with NaN values for values not included.
def national_yearly_data(frame):
    national_data = frame.drop(columns=META_COLUMNS, axis=1).sum()

    d = {'United States': national_data}

    df = pd.DataFrame(data=d).reset_index(names='Date')
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')

    dates = ['2020-01-22','2020-12-31','2021-01-01','2021-12-31','2022-01-01','2022-12-31','2023-01-01','2023-07-23']

    years = ['2020', '2021', '2022', '2023']

    i = 0
    arrays = []
    for year in years:
        data = df.loc[dates[i] : dates[i+1]].reset_index()
        data['Yearless'] = data['Date'].astype(str).str.slice(5)
        data.drop(columns=['Date'], inplace=True)
        data.rename(columns={'United States':year, 'Yearless':'Date'}, inplace=True)
        arrays.append(data)
        i+=2
    
    dff = arrays[0].drop(columns='2020', axis=1)
    for array in arrays:
        dff = dff.merge(array, how='outer', on='Date')

    return dff

national_yearly_data(DEATHS)

,Date,2020,2021,2022,2023
0,01-01,NaN,357978.0,819616.0,1068813.0
1,01-02,NaN,361430.0,820029.0,1068813.0
2,01-03,NaN,363775.0,822216.0,1068313.0
3,01-04,NaN,366113.0,824396.0,1068502.0
4,01-05,NaN,369998.0,826187.0,1069200.0
...,...,...,...,...,...
361,12-27,340430.0,812968.0,1066808.0,NaN
362,12-28,344373.0,814879.0,1067030.0,NaN
363,12-29,348147.0,816757.0,1067244.0,NaN
364,12-30,351694.0,817902.0,1068810.0,NaN
